<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_02_notebook_modeles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display
import os
import nltk
import pickle
import re
from collections import Counter

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, average_precision_score, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
data = pd.read_pickle("data/data_tokenized_sw_free_v2.pkl")
print(f"Taille du jeu de données : {data.shape}")
data.head()

Taille du jeu de données : (296944, 4)


,original_title,original_body,post,tags
Id,,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,node taking hour employee recordsgroovy xmlbe...,[javascript]
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,python usingi based ge ge begin create called...,[python]
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,object allocated allocate constructremoved des...,[c++]
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


# Partie non supervisée

In [ ]:
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt

df = data.copy()
df.head()

,original_title,original_body,post,tags
Id,,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,node taking hour employee recordsgroovy xmlbe...,[javascript]
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,python usingi based ge ge begin create called...,[python]
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,object allocated allocate constructremoved des...,[c++]
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [ ]:
texts = df.post.str.split().to_list()

id2word = corpora.Dictionary()
vocabulary = pickle.load(open("data/list_vocab.pkl", "rb"))
id2word.add_documents([[x] for x in vocabulary])

bow_corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = [tfidf[text] for text in bow_corpus]

In [ ]:
lda_model = LdaMulticore(corpus = tfidf_corpus,
                    id2word = id2word,
                    num_topics = 20, # nombre de tags retenus
                    random_state = 42,
                    passes = 10,
                    workers = 7,
                    minimum_probability = 0)

In [ ]:
model.log_perplexity(tfidf_corpus)

-6.672111449137571

In [ ]:
coherencemodel = CoherenceModel(model = model, 
                                texts = texts,
                                dictionary = id2word, 
                                coherence = 'c_v')

coherencemodel.get_coherence()

0.42942308401786605

In [ ]:
from pprint import pprint

model_topics = lda_model.show_topics(formatted=False)
pprint(lda_model.print_topics(num_words=30))

[(0,
  '0.034*"convert" + 0.024*"read" + 0.020*"string" + 0.019*"key" + '
  '0.018*"dict" + 0.016*"exple" + 0.014*"json" + 0.013*"format" + 0.013*"ve" + '
  '0.012*"variable" + 0.012*"tried" + 0.012*"object" + 0.012*"don" + '
  '0.012*"question" + 0.012*"trying" + 0.012*"gl" + 0.011*"extension" + '
  '0.011*"help" + 0.011*"thanks" + 0.011*"image" + 0.010*"able" + 0.010*"time" '
  '+ 0.010*"create" + 0.010*"result" + 0.010*"multiple" + 0.010*"using" + '
  '0.010*"buffer" + 0.009*"print" + 0.009*"following" + 0.009*"text"'),
 (1,
  '0.059*"andro" + 0.041*"val" + 0.036*"fun" + 0.035*"activity" + '
  '0.033*"intent" + 0.025*"layout" + 0.024*"overre" + 0.019*"dp" + '
  '0.019*"permission" + 0.018*"fragment" + 0.016*"view" + 0.015*"recyclerview" '
  '+ 0.014*"savedinstancestate" + 0.013*"context" + 0.013*"java" + '
  '0.013*"mainactivity" + 0.012*"tag" + 0.011*"wrap" + 0.011*"androx" + '
  '0.010*"item" + 0.010*"parent" + 0.009*"content" + 0.009*"xmlns" + '
  '0.008*"com" + 0.008*"public" + 

In [ ]:
def format_topics_sentences(ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), 
                                                                  round(prop_topic,4),
                                                                  topic_keywords]),
                                                       ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model,
                                                  corpus=tfidf_corpus)
df_topic_sents_keywords

df_topic_sents_keywords.index = df.index
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords
Id,,,
65526399,6.0,0.6450,"py, python, install, site, package, usr, run, ..."
65526401,2.0,0.7756,"self, def, view, print, func, var, nil, swiftu..."
65526407,16.0,0.4496,"request, www, url, php, socket, server, page, ..."
65526419,11.0,0.4592,"std, foo, int, template, operator, vector, cpp..."
65526447,9.0,0.8365,"react, component, const, div, prop, props, use..."


In [ ]:
df_tmp = df_topic_sents_keywords.copy()
df_tmp.sort_values(by = ["Perc_Contribution"], ascending = False, inplace = True)
df_tmp.drop_duplicates(subset = ["Dominant_Topic"], inplace = True)
df_tmp = df_tmp.join(df[["tags"]])

df_tmp[["Dominant_Topic", "Topic_Keywords", "tags"]]

,Dominant_Topic,Topic_Keywords,tags
Id,,,
61372980,14.0,"child, context, wget, icon, dart, firestore, b...",[flutter]
60090428,17.0,"node, js, module, react, npm, modules, angular...",[reactjs]
67265453,3.0,"public, org, java, microsoft, string, method, ...",[c#]
59567602,19.0,"aws, ec, access, role, bucket, cluster, resour...","[node.js, amazon-web-services]"
61418553,7.0,"await, const, log, async, err, message, respon...",[node.js]
62308701,8.0,"google, studio, andro, flutter, play, device, ...","[android, flutter]"
63313952,1.0,"andro, val, fun, activity, intent, layout, ove...",[android]
59587066,11.0,"std, foo, int, template, operator, vector, cpp...",[c++]
60908554,13.0,"div, px, span, style, td, li, img, button, col...",[javascript]


In [ ]:
df_tmp = df.copy()
df_tmp.post = df_tmp.post.str.split()

def predict_unsupervised_tags(text, lda_model, id2w):
    """
    Predict tags of a preprocessed text
    
    Args:
        text(list): preprocessed text
        
    Returns:
        relevant_tags(list): list of tags
    """
    corpus_new = id2w.doc2bow(text)
    topics = lda_model.get_document_topics(corpus_new)
    
    #find most relevant topic according to probability
    relevant_topic = topics[0][0]
    relevant_topic_prob = topics[0][1]
    
    for i in range(len(topics)):
        if topics[i][1] > relevant_topic_prob:
            relevant_topic = topics[i][0]
            relevant_topic_prob = topics[i][1]
            
    #retrieve associated to topic tags present in submited text
    potential_tags = lda_model.get_topic_terms(topicid=relevant_topic, topn=20)
    
    relevant_tags = [id2w[tag[0]] for tag in potential_tags if id2w[tag[0]] in text]
    
    return relevant_tags

for i in range(30):
    print(f"Liste de tags à prédire : {df.sample(30, random_state = 0).tags.iloc[i]}\n")
    print(f"Liste des tags prédits par le modèle non supervisé: {predict_unsupervised_tags(df_tmp.sample(30, random_state = 0).post.iloc[i],lda_model,id2word)}\n\n")

Liste de tags à prédire : ['javascript', 'html', 'css']

Liste des tags prédits par le modèle non supervisé: ['selected']


Liste de tags à prédire : ['angular']

Liste des tags prédits par le modèle non supervisé: []


Liste de tags à prédire : ['javascript', 'angular']

Liste des tags prédits par le modèle non supervisé: ['selector', 'vo', 'html']


Liste de tags à prédire : ['python', 'python-3.x']

Liste des tags prédits par le modèle non supervisé: ['gr']


Liste de tags à prédire : ['android', 'flutter']

Liste des tags prédits par le modèle non supervisé: ['flutter']


Liste de tags à prédire : ['python', 'python-3.x']

Liste des tags prédits par le modèle non supervisé: ['version']


Liste de tags à prédire : ['amazon-web-services']

Liste des tags prédits par le modèle non supervisé: ['lication']


Liste de tags à prédire : ['python', 'python-3.x']

Liste des tags prédits par le modèle non supervisé: ['author']


Liste de tags à prédire : ['php']

Liste des tags prédits par le

In [ ]:
df_res_unsup = data.copy()
df_res_unsup["tags_predicted_unsupervised"] = df_res_unsup.post.map(lambda x: predict_unsupervised_tags(x.split(), 
                                                                                            lda_model,
                                                                                            id2word))
pickle.dump(df_res_unsup, open("resultats/df_unsupervised.pkl", "wb"))
df_res_unsup.head()

,original_title,original_body,post,tags,tags_predicted_unsupervised
Id,,,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,node taking hour employee recordsgroovy xmlbe...,[javascript],[person]
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,python usingi based ge ge begin create called...,[python],[]
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,remove ver powered whmcomplete solution whmcsi...,"[php, css]",[version]
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,object allocated allocate constructremoved des...,[c++],[]
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,react route change createbrowserhistory blockr...,"[javascript, reactjs]",[]


In [ ]:
df_res_unsup = pickle.load(open("resultats/df_unsupervised.pkl", "rb"))
df_res_unsup.head()

,original_title,original_body,post,tags,tags_predicted_unsupervised
Id,,,,,
65526399,Groovy script code to append xml node taking 1...,<p>Below code is working but taking 15+ hours ...,node taking hour employee recordsgroovy xmlbe...,[javascript],[person]
65526401,How do I make a class instance using user input?,<p>I am making a text based adventure game in ...,python usingi based ge ge begin create called...,[python],[]
65526407,"How to remove Branding WHMCS Ver 8.1 ""Powered ...",<p>I am a newbie and this is the first time ev...,remove ver powered whmcomplete solution whmcsi...,"[php, css]",[version]
65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,object allocated allocate constructremoved des...,[c++],[]
65526447,React Router v5.2 - Blocking route change with...,<p>My app has two pages: <code>Step1</code> an...,react route change createbrowserhistory blockr...,"[javascript, reactjs]",[]


In [ ]:
mlb_unsup = MultiLabelBinarizer()
mlb_unsup.fit(df_res_unsup.tags_predicted_unsupervised)

tags_detectes = set(mlb.classes_) & set(mlb_unsup.classes_)

print(f"Nombre de tags à prédire : {len(mlb.classes_)}\n")
print(f"Tags détectés par notre modèle non supervisé : {tags_detectes}\n")
print(f"Nombre de tags manquants : {len(mlb.classes_) -  len(tags_detectes)}\n")
print(f"Nombre de tags en trop : {len(mlb_unsup.classes_) - len(tags_detectes)}")

Nombre de tags à prédire : 20

Tags détectés par notre modèle non supervisé : {'html', 'flutter'}

Nombre de tags manquants : 18

Nombre de tags en trop : 312


In [ ]:
precision = precision_score(mlb.transform(df_res_unsup.tags.to_list()), 
                            mlb.transform(df_res_unsup.tags_predicted_unsupervised.to_list()),
                            average = "macro",
                            zero_division = False)

print(f"precision score : {round(precision, 3)}")

acc = accuracy_score(mlb.transform(df_res_unsup.tags.to_list()),
                     mlb.transform(df_res_unsup.tags_predicted_unsupervised.to_list()))

print(f"accuracy score : {round(acc, 3)}")

precision score : 0.064
accuracy score : 0.007
